In [2]:
import sys
sys.path.append("../../src")
import numpy as np
import pandas as pd
import lightgbm
# from utils import ndcg_calculator
import datetime as dt

In [3]:
def dataload(path:str='../../data/'):

    test_answer_week = pd.read_parquet(path + "test_answer_week.parquet")
    test_answer_month = pd.read_parquet(path + "test_answer_month.parquet")

    df_train_week = pd.read_parquet(path + "train_week.parquet")
    df_train_month = pd.read_parquet(path + "train_month.parquet")

    sample_sumbission_week = pd.read_parquet(path + "sample_sumbission_week.parquet")
    sample_sumbission_month = pd.read_parquet(path + "sample_sumbission_month.parquet")

    df_train_week.sort_values(by='log_dt', inplace=True)
    df_train_month.sort_values(by='log_dt', inplace=True)
    
    return test_answer_week, test_answer_month, df_train_week, df_train_month, sample_sumbission_week, sample_sumbission_month

In [4]:
path='/kaggle/input/lg-train-test/'

test_answer_week, test_answer_month, \
df_train_week, df_train_month, \
sample_sumbission_week, sample_sumbission_month = dataload(path)

# explore data for candidate

분리된 데이터에서 재시청, 신규 시청 등이 어떻게 일어났는지 확인한다.    
week, month에서 모두 확인한다. 

> 재시청된 item list 추출 (7주/1주 . 1주간 다시 재시청된 영상 3585개

week 결론     
- 25%의 유저가 새로운 영상을 탐색한다.    

- 20.58%의 유저가 1개 이상의 영상을 재시청한다.


month 결론
- 77.52%의 유저가 새로운 영상을 탐색한다. 

- 44.92%의 유저가 1개 이상의 영상을 재시청한다.

In [18]:
history=pd.read_parquet("/kaggle/input/history-for-lgbm/df_train.parquet")
history = feature_engineering(history)
# history['log_date'] = pd.to_datetime(history['log_date'])
# history

In [110]:
import datetime
df_train_week = history[history['week'] <= 7]
test_answer_week = history[history['week'] > 7]

# week trian : 7주치 데이터에서 시청된 영상 set 추출
week_album_trian_set = df_train_week.groupby('profile_id')['album_id'].apply(set)
# week의 마지막 1주 데이터에서 조회된 영상 set 추출
week_album_test_set = test_answer_week.groupby('profile_id')['album_id'].apply(set)

#유저-상품 단위의 딕셔너리 정의
# 유저가 영상을 마지막 일주일 이전에 조회했는지 혹은 마지막 일주일동안 조회했는지를 표시한다.
# 7주차 내에 시청한 영상은 'clicked'라고 표기한다.
profile_album_dict = {}
for profile_id, album_ids in week_album_trian_set.items():
    profile_album_dict[profile_id] = {}
    for album_id in album_ids:
        profile_album_dict[profile_id][album_id] = 'clicked'

# 7주간 조회한 영상은 'clicked', 마지막 1주에만 조회한 영상은 'new', 두 기간동안 모두 조회한 영상(재시청)은 'both'라고 표기
for profile_id, album_ids in week_album_test_set.items():
    # 마지막 일주일간 시청기록이 있는 유저인지를 체크한다.
    if profile_id in profile_album_dict:
        for album_id in album_ids:
            # 시청한 적 있는 상품인지를 체크한 뒤, 상태를 표기한다.
            if album_id in profile_album_dict[profile_id]:
                profile_album_dict[profile_id][album_id] = 'both'
            else:
                profile_album_dict[profile_id][album_id] = 'new'
    # 마지막 일주일간 시청기록이 없는 유저라면 모두 'new'로 표기한다.
    else:
        profile_album_dict[profile_id] = {}
        for album_id in album_ids:
            profile_album_dict[profile_id][album_id] = 'new'
            
# 'clicked', 'new', 'both'를 유저별로 탐색하여 데이터 프레임을 생성한다.
columns = ['profile_id', 'clicked', 'new', 'both']
df_order_info = pd.DataFrame(columns=columns)
# 데이터 프레임 생성
for profile_id in profile_album_dict:
    clicked = 0
    new = 0
    both = 0
    # 딕셔너리의 상태(clicked, new, both)를 체크하여 데이터 프레임에 append 할 수 있는 형태로 처리한다.
    for clicked in profile_album_dict[profile_id]:
        status = profile_album_dict[profile_id][clicked]
        if status == 'clicked':
            clicked += 1
        elif status == 'new':
            new += 1
        else:
            both += 1
    # df_order_info에 데이터를 append한다.
    row = [profile_id, clicked, new, both]
    series = pd.Series(row, index=columns)
    df_order_info = df_order_info.append(series, ignore_index=True)
    
df_order_info.head()

,profile_id,clicked,new,both
0,3,34,0,0
1,5,317,4,2
2,7,344,0,0
3,12,357,2,0
4,16,125,0,0


7주간 본 영상 개수 = clicked   
1주간 새로 본 영상 개수 = new   
8주 중 중복시청한 영상 개수 = both

In [59]:
# 데이터 프레임에서 clicked 1 이상이면서, new가 1 이상인 유저 수 
print(f"마지막 일주일간 기존에 본 적 없는 영상을 시청한 유저 : {df_order_info[(df_order_info['clicked']>0) & (df_order_info['new']>0)].shape[0]}명")

# 데이터 프레임에서 both가 1 이상인 유저 수 
print(f"재시청한 영상이 있는 유저 : {df_order_info[df_order_info['both'] > 0].shape[0]}명")

# # 중복시청을 한 적 없는 유저
# print(f"중복 시청을 하지 않은 유저 : {df_order_info[df_order_info['both'] == 0].shape[0]}명")

'''
마지막 일주일간 기존에 본 적 없는 영상을 시청한 유저 : 2188명
재시청한 영상이 있는 유저 : 1710명
'''

마지막 일주일간 기존에 본 적 없는 영상을 시청한 유저 : 2188명
재시청한 영상이 있는 유저 : 1710명


In [23]:
# 만약 새로운 영상을 시청한다면 얼마나 많은 종류의 새로운 영상을 시청하는지 탐색한다.
print(df_order_info['new'].value_counts()[1:].describe())

count     47.000000
mean      46.829787
std       93.276196
min        1.000000
25%        1.500000
50%        7.000000
75%       43.500000
max      451.000000
Name: new, dtype: float64


In [89]:
# 데이터 프레임에서 clicked 1 이상이면서, new가 1 이상인 유저 수 
print(f"마지막 4주간 기존에 본 적 없는 영상을 시청한 유저 : {df_order_info[(df_order_info['clicked']>0) & (df_order_info['new']>0)].shape[0]}명")

# 데이터 프레임에서 both가 1 이상인 유저 수 
print(f"재시청한 영상이 있는 유저 : {df_order_info[df_order_info['both'] > 0].shape[0]}명")

'''
마지막 4주간 기존에 본 적 없는 영상을 시청한 유저 : 6443명
재시청한 영상이 있는 유저 : 3733명
'''

마지막 4주간 기존에 본 적 없는 영상을 시청한 유저 : 6443명
재시청한 영상이 있는 유저 : 3733명


In [90]:
# 만약 새로운 영상을 시청한다면 얼마나 많은 종류의 새로운 영상을 시청하는지 탐색한다.
print(df_order_info['new'].value_counts()[1:].describe())

count    185.000000
mean      34.837838
std       80.276937
min        1.000000
25%        1.000000
50%        4.000000
75%       23.000000
max      589.000000
Name: new, dtype: float64


In [122]:
## 7주간 시청된 것들 중, 마지막 1주동안 재시청된 아이템 리스트를 뽑는다.

profile_album_dict = {}
for profile_id, album_ids in week_album_trian_set.items():
    profile_album_dict[profile_id] = {}
    for album_id in album_ids:
        profile_album_dict[profile_id][album_id] = 'clicked'

MP_reviewed = []
for profile_id, album_ids in week_album_test_set.items():
    if profile_id in profile_album_dict:
        for album_id in album_ids:
            # 시청한 적 있는 상품인지를 체크한 뒤, 상태를 표기한다.
            if album_id in profile_album_dict[profile_id]:
                MP_reviewed.append(album_id)
                
len(set(MP_reviewed))  # 재시청된 아이템 리스트 :: 3585개 아이템

3585